Our first method will use simple counting i.e. count the number of times each character is predicted based on two characters provided e.g. the name jane would defines n as the next characters after 'ja' so that would add to 'n' probability of being the next character 

In [ ]:
import math
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline 
%pip install torch torchvision
import torch

In [ ]:
# Read in words
words = open('names.txt', 'r').read().splitlines()
words[:10]


In this instance, we are only looking at three characters. Two input characters, and the character we are predicting next.

e.g. what characters are likely to follow r

we also now what names are likely to start and finish at


We then do a simple count of most prominent trigrams

How many examples do we get from emma?

<S>e -> m
em -> m
mm -> a
ma -> <E>

In [ ]:
t = {}
for w in words:
    chs = ['<S>'] + list(w) + ['<E>'] # hallucinate a start character and end character 
    for pair, ch2 in zip(zip(chs, chs[1:]), chs[2:]):
        trigram  = (''.join(pair), ch2)
        t[trigram] = t.get(trigram, 0) + 1

Now lets get the count of each combo and sort. we'll see that ah followed by ending character occurs the

In [ ]:
sorted_t = sorted(t.items(), key = lambda kv: -kv[1])
sorted_t[:10]

We need to create a lookup of integers to pairs so we can have an index. 
Basically we need an integer ref for all our points in our torch, that means each character pair needs an integer value, and each character needs a value e.g. ac = [100] and if it predicts d as next letter it would be axis point [100, 3]. Therefore each time this is shown, we add +1, allowing us to count the number of occurences 

If I understand this correctly, we therefore need two integer lookups: 
-
Taking our char pairs as the 'y' axis - we need 0:len(unique-pairs)  i.e. 0-600 and then our single character axis being x axis, we need 28 chars (alphabet including <S> and <E>)

0   |------------------------------| 28
    |<S>aa   <S>ab  <S>ac
    |aaa     aab    aac
    |..
    |..
    |..
784 |zaa zab

Lets get the character string to integer index for our 'y-axis' which is a - z and our <S> and <E> characters

In [ ]:
# returns all the set of lowercase characters
chars = sorted(set(''.join(words)))
char_to_integer = {s:i for i,s in enumerate(chars)}
char_to_integer['<S>'] = 26
char_to_integer['<E>'] = 27
print(list(char_to_integer)[:10])

Lets convert our trigram to an array. We need to deduce how many rows we have, so it should be all the combinations of characters against our list of 28 chars.

28 characters consist of 26 alphabet characters plus our <S> and <E> characters. So as we can pair same characters together e.g. -> a,a and with the other 27 chars, there is a total of 28*28 combinations -> 784 (minus some impossible combos e.g. <S><E> a<S> <E>s)

Note we can remove all pairs that begin with <E> as its not possible to start on an end character

In [ ]:
import string

# Create a list of all lowercase letters plus <S> and <E>
letters = list(string.ascii_lowercase) + ["<S>", "<E>"]

# Create a list of tuples where the first element is a pair of characters and the second element is an integer index
pairs_string_to_integer = [(a+b, i) for i, (a, b) in enumerate((x, y) for x in letters for y in letters)]

## And finally convert this to dictionary so we can do lookups
pairs_string_to_integer = dict(pairs_string_to_integer)
pairs_string_to_integer['fa']

# Remove all keys where <S> is the second character
pairs_string_to_integer = {key: value for key, value in pairs_string_to_integer.items() if not key.endswith('<S>')}

# Remove all keys where <E> is the first character
pairs_string_to_integer = {key: value for key, value in pairs_string_to_integer.items() if not key.startswith('<E>')}

# Remove <S><E> as not possible
pairs_string_to_integer = {key: value for key, value in pairs_string_to_integer.items() if not key == ("<S><E>")}

# Reset the indices
pairs_string_to_integer = {key: i for i, (key, value) in enumerate(pairs_string_to_integer.items())}

print(pairs_string_to_integer)


In [ ]:
## Creates a 784 * 28 tensor that is empty initially
import torch

N = torch.zeros((len(pairs_string_to_integer), 28), dtype=torch.int32) # 28*28 = 784 for different pair combos
N[:5]

Now add our counts of each character to the tensor i.e. all counts of aab -> zzz occuring

Each time e.g. ab pair with pred c comes up, we add a count to the co-ordinates in the tensor. As a result, we get the probability of each pair and next character occuring

In [ ]:
for w in words:
    chs = ['<S>'] + list(w) + ['<E>'] # hallucinate a start character and end character 
    for pair, ch2 in zip(zip(chs, chs[1:]), chs[2:]):
        trigram  = (''.join(pair), ch2) #  e.g. ('<S>e', 'm')
        pairs_combined = trigram[0] #  e.g '<S>e'
        print
        ix1 = pairs_string_to_integer[pairs_combined]
        ix2 = char_to_integer[ch2]
        N[ix1, ix2] += 1  

In [ ]:
N[10]

In [ ]:
%pip install matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

Create an inverse of string to integer, this way we can look up our characters based on the cordinates in our tensor N

In [ ]:
char_itos = {i:s for s,i in char_to_integer.items()}
pair_itos = {i:s for s,i in pairs_string_to_integer.items()}
pair_itos[100]

In [ ]:
# TODO: FIX THIS
plt.figure(figsize=(16,16))
plt.imshow(N, cmap='Blues')
for i in range(len(pair_itos)):
    for j in range(len(char_itos)):
        chstr = pair_itos[i] + char_itos[j]
        plt.text(j, i, chstr, ha="center", va="bottom", color='gray')
        plt.text(j, i, N[i, j].item(), ha="center", va="top", color="gray")
plt.axis('off')

Convert the counts to proabilities of what character a pair will predict next. We add 0.01 to avoid predicting a pair which was 0 occurrences, that would result in / 0.

In [ ]:
P = (N+0.01).float()  
P /= P.sum(1, keepdims=True)
P[1]

For sampling we will use torch.multinomial and a generator object to make everything deterministic 

This line of code is using PyTorch, a popular machine learning library in Python. 

The `torch.Generator()` is an object that holds the state of the random number generator. You can think of it as a container for the algorithm that produces pseudo-random numbers.

The `manual_seed()` function is used to set the seed for generating random numbers. This ensures that the random numbers generated are deterministic, meaning if you use the same seed, you will get the same sequence of random numbers. This is useful for debugging and testing purposes, as it allows for reproducibility in your code.

In [ ]:
g = torch.Generator().manual_seed(2147483647)

Logic -> We predict the 3rd character from the beginning two e.g. em -> a. 
We then strip the first character and append our 3rd character for the next prediction i.e. ma -> b

- Starting off the prediction, we need to start on <S>(x) character

In [ ]:
start_chars = []
for key, value in pair_itos.items():
    if '<S>' in str(value):
        start_chars.append((key, value))
start_chars[:10]

Note - When we run through predictions, we need to predict like ab -> c -> bc -> d -> cd -> e
Therefore we need to combine the second character in the pair with the predicted value. This is demonstrated below.

Note: -1 gets the last character from a string

In [ ]:
ix_pair = pair_itos[10][-1] + char_itos[10] 
ix_pair

pairs_string_to_integer[ix_pair]
P[45]

In [ ]:
import random
g = torch.Generator().manual_seed(214748347)

for i in range(10):
    # Select a random value from start_chars
    out = []
    start_ix = random.choice(start_chars) # Gets all pairs that start with <S> 
    ix_pair = start_ix[0] # Gets integer reference for pair e.g. fe -> 45
    out.append(start_ix[1]) # Append the first start characters
    while True:
        p = P[ix_pair]
        iy_pred_reference = torch.multinomial(p, num_samples=1, replacement=True, generator=g).item()
        out.append(char_itos[iy_pred_reference])
        if '<E>' in char_itos[iy_pred_reference]:
            break
        # Combines last letter of pair and predicted char e.g. xy predicts z  so ix_pair now equals  = yz 
        # if '<S>' in pair_itos[ix_pair]:

        ix_pair_str = pair_itos[ix_pair][-1] + char_itos[iy_pred_reference] 
        
        # Convert ix back to an integer reference
        ix_pair = pairs_string_to_integer[ix_pair_str]


    print(''.join(out))


Now lets look at calculating the loss function to check the quality of the model

GOAL: maximize likelihood of the data w.r.t. model parameters (statistical modeling)
equivalent to maximizing the log likelihood (because log is monotonic)
equivalent to minimizing the negative log likelihood
equivalent to minimizing the average negative log likelihood

log(a*b*c) = log(a) + log(b) + log(c)

In [ ]:
log_likelihood = 0.0
n = 0 
for w in words:
    chs = ['<S>'] + list(w) + ['<E>'] # hallucinate a start character and end character 
    for pair, ch in zip(zip(chs, chs[1:]), chs[2:]):
        pair_combined = pair[0] + pair[1]
        ix1 = pairs_string_to_integer[pair_combined]
        ix2 = char_to_integer[ch]
        prob = P[ix1, ix2]
        logprob = torch.log(prob)
        log_likelihood += logprob
        n +=1 
        # print(f'{pair}{ch}: {prob:.4f} {logprob:.4f}')
print(f'{log_likelihood}')

negative_log_liklihood = -log_likelihood
print(f'{negative_log_liklihood=}')
print(f'{negative_log_liklihood/n}') # Average log likelihood


In [ ]:
words[:1]

# Method 2  - Neural Net and gradient descent

Lets now do this via a neural net - first we take the first name 'emma' as an example

In [ ]:
# Create a training set of all the trigrams (ab,c)
xs, ys = [], []

for w in words[:1]:
    chs = ['<S>'] + list(w) + ['<E>'] # hallucinate a start character and end character 
    for pair, ch in zip(zip(chs, chs[1:]), chs[2:]):
        pair_combined = pair[0] + pair[1]
        ix1 = pairs_string_to_integer[pair_combined]
        ix2 = char_to_integer[ch]
        xs.append(ix1)
        ys.append(ix2)
xs = torch.tensor(xs)
ys = torch.tensor(ys)
xs


tensor([706, 120, 336, 324]) = <S>e <em> <mm> <ma>

In [ ]:
pair_itos[324]

In [ ]:
ys
len(pairs_string_to_integer)

In [ ]:
import torch.nn.functional as F

xenc = F.one_hot(xs, num_classes=len(pairs_string_to_integer)).float()
xenc

In [ ]:
xenc.shape 

We can see the different pairs from 'emma' denoted in the below graph. 

In [ ]:
plt.imshow(xenc, aspect=20/1)

In [ ]:
len(xs)
len(ys)

Create a neuron, still focusing just on emma

In [ ]:
xenc.shape

In [ ]:
# weights
W = torch.randn(len(pairs_string_to_integer),28)

# Multiple encodings by weight, feed all our inputs into the neuron i.e. w 
xenc @ W # 4 * 27

Above, we are getting ->  (4, 728) @ [728, 4] -> (4, 27).

The above is telling, for every one of 27 neurons created, what is the firing rate of each neuron, on every one of those 4 examples. Below, (xenc @ W)[3,13] is telling us the firing rate of the 13th neuron looking at the 3rd input. 

This is achieved by multipled 3rd input (xs) by column 13


In [ ]:
# Now lets get exponent to make it everything positive. We can interpret these as counts in the neural net. 
# Log-counts - logics
logits = xenc @ W # log-counts

counts = logits.exp() # equivalent N

# normalise to get the probs
# counts divided by total of each row gives us the prob
probs = counts / counts.sum(1, keepdim=True)
probs

What have we achieved? for every one of our 4 examples, we now have a row that came out of a neural net. And because we transformed, they are now probabilities.

-- For all the operations we conducted,  interpret logits to be log counts, we exponentiate to get something that looks like log counts, then normalise to get probability. All of these are differential operations ( we can get derivative), meaning we can back propogate through and get out prob distributions
logits = xenc @ W # log-counts
counts = logits.exp() # equivalent N
probs = counts / counts.sum(1, keepdim=True)

In [ ]:
probs.shape

In [ ]:
nlls = torch.zeros(4)
for i in range(4):
  # i-th bigram:
  x = xs[i].item() # input character index
  y = ys[i].item() # label character index
  print('--------')
  print(f'bigram example {i+1}: {pair_itos[x]}{char_itos[y]} (indexes {x},{y})')
  print('input to the neural net:', x)
  print('output probabilities from the neural net:', probs[i])
  print('label (actual next character):', y)
  p = probs[i, y]
  print('probability assigned by the net to the the correct character:', p.item())
  logp = torch.log(p)
  print('log likelihood:', logp.item())
  nll = -logp
  print('negative log likelihood:', nll.item())
  nlls[i] = nll

print('=========')
print('average negative log likelihood, i.e. loss =', nlls.mean().item())


# ------- Optimization -----

In [ ]:
# Inputs
xs

In [ ]:
ys

In [ ]:
# randomly initialize 27 neurons' weights. each neuron receives 27 inputs
g = torch.Generator().manual_seed(2147483647)
W = torch.randn((len(pairs_string_to_integer),28), generator=g, requires_grad=True)

In [ ]:
xenc = F.one_hot(xs, num_classes=len(pairs_string_to_integer)).float()
logits = xenc @ W # predict log-counts
counts = logits.exp() # counts, equivalent to N
probs = counts / counts.sum(1, keepdims=True) # probabilities for next character
# btw: the last 2 lines here are together called a 'softmax'
loss = -probs[torch.arange(4), ys].log().mean()
loss
xenc

In [ ]:
#backward pass
W.grad = None # reset gradiants
# loss.backward backpropogates through probs and counts and logits to give gradients for our original tensir
loss.backward()

In [ ]:
W.grad[706]

Perform gradient descent manuallt below

In [ ]:
W.data += -0.1 * W.grad
W.data

# --------- !!! OPTIMIZATION !!! yay, but this time actually --------------

In [ ]:
# Create a training set of all the trigrams (ab,c)
xs, ys = [], []

for w in words:
    chs = ['<S>'] + list(w) + ['<E>'] # hallucinate a start character and end character 
    for pair, ch in zip(zip(chs, chs[1:]), chs[2:]):
        pair_combined = pair[0] + pair[1]
        ix1 = pairs_string_to_integer[pair_combined]
        ix2 = char_to_integer[ch]
        xs.append(ix1)
        ys.append(ix2)
xs = torch.tensor(xs)
ys = torch.tensor(ys)
num = xs.nelement()
print('number of examples: ', num)


# # initialize the 'network'
g = torch.Generator().manual_seed(2147483647)
W = torch.randn((len(pairs_string_to_integer),28), generator=g, requires_grad=True)

In [ ]:
# gradient descent
for k in range(140):
  # forward pass
  xenc = F.one_hot(xs, num_classes=len(pairs_string_to_integer)).float()
  logits = xenc @ W # predict log-counts
  counts = logits.exp() # counts, equivalent to N
  probs = counts / counts.sum(1, keepdims=True) # probabilities for next character
  # btw: the last 2 lines here are together called a 'softmax'
  loss = -probs[torch.arange(num), ys].log().mean() + 0.01*(W**2).mean()
  print(loss.item())
  
  # backward pass
  W.grad = None # set to zero the gradient
  loss.backward()
  
  # update
  W.data += -50 * W.grad

In [ ]:
# finally, sample from the 'neural net' model
g = torch.Generator().manual_seed(2147483647)

for i in range(5):
  
  out = []
  start_ix = random.choice(start_chars) # Gets all pairs that start with <S> 
  ix_pair = start_ix[0] # Gets integer reference for pair e.g. fe -> 45
  out.append(start_ix[1]) # Append the first start characters
  while True:
    # ----------
    # BEFORE:
    #p = P[ix]
    # ----------
    # NOW:
    xenc = F.one_hot(torch.tensor([ix_pair]), num_classes=len(pairs_string_to_integer)).float()
    logits = xenc @ W # predict log-counts
    counts = logits.exp() # counts, equivalent to N
    p = counts / counts.sum(1, keepdims=True) # probabilities for next character
    # hack -> Set the probability of <S> to zero
    s_index = char_to_integer['<S>']
    p[0, s_index] = 0  # TODO: make this cleaner
    p = p / p.sum(1, keepdims=True)  # re-normalize the probabilities
    # ----------
    iy_pred_reference = torch.multinomial(p, num_samples=1, replacement=True, generator=g).item()
    out.append(char_itos[iy_pred_reference])

    if '<E>' in char_itos[iy_pred_reference]:
      break
    # Combines last letter of pair and predicted char e.g. xy predicts z  so ix_pair now equals  = yz 
    ix_pair_str = pair_itos[ix_pair][-1] + char_itos[iy_pred_reference] 
    # Convert ix back to an integer reference
    ix_pair = pairs_string_to_integer[ix_pair_str]
  print(''.join(out))